# Create a Spark session and load csv data #

In [1]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.getOrCreate()

# Load the Stack Overflow survey response CSV file into a DataFrame
soDf = spark.read.csv("C:\\Users\\Phil.Austin\\OneDrive - Telefonica Tech UK Limited\\survey_results_public.csv", header=True, inferSchema=True)

# Show the first few rows of the DataFrame
soDf.show()

+----------+--------------------+------------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+------------+--------------------+-------+-----------------+----------+----------+-----------+--------------------+--------+---------+----------------------+----------------------+--------------------+----------------------+----------------------+--------------------+----------------------+----------------------+--------------------+----------------------+----------------------+--------------------+----------------------+----------------------+-----------------+----------------------+----------------------+--------------------+-----------------------+-----------------------+--------------------+----------------------------+----------------------------+---------------------+--------------------+---------------------+------------------------------+----------------------------

In [2]:
soDf.createOrReplaceTempView("so_data")

grpDf = spark.sql("""
                    SELECT 
                        Age, 
                        COUNT(*) AS NumberOfResponses 
                    FROM so_data 
                    GROUP BY Age 
                    ORDER BY COUNT(*) DESC
                """)

grpDf.show()

+------------------+-----------------+
|               Age|NumberOfResponses|
+------------------+-----------------+
|   25-34 years old|            23911|
|   35-44 years old|            14942|
|   18-24 years old|            14098|
|   45-54 years old|             6249|
|   55-64 years old|             2575|
|Under 18 years old|             2568|
| 65 years or older|              772|
| Prefer not to say|              322|
+------------------+-----------------+



# Salaries are in different currencies #

## Which you can see in Data Wrangler... ##


In [3]:
#Load currency conversions
currencyConvDf = spark.read.csv("average_csv_2024-3.csv", header=True, inferSchema=True)

currencyConvDf.show()


+----------+------------------+-------------+---------------------------------+---------------------+
|   Country|  Unit Of Currency|Currency Code|Sterling value of Currency Unit £|Currency Units per £1|
+----------+------------------+-------------+---------------------------------+---------------------+
| Abu Dhabi|            Dirham|          AED|                           0.2173|               4.6012|
|   Albania|               Lek|          ALL|                           0.0081|             122.9802|
|   Algeria|             Dinar|          DZD|                           0.0059|             169.8818|
|    Angola|      Readj Kwanza|          AOA|                           0.0011|             944.5533|
|   Antigua|E Caribbean Dollar|          XCD|                           0.2955|               3.3846|
| Argentina|              Peso|          ARS|                           0.0019|             528.1742|
|   Armenia|              Dram|          AMD|                            0.002|   

# Add a Currency Code column to the SO dataframe #

In [4]:
from pyspark.sql import functions as F

# Add a new column 'CurrencyCode' to soDf that contains the first three letters of 'Currency'
soDf = soDf.withColumn("CurrencyCode", F.expr("substring(Currency, 1, 3)"))

soDf.filter(soDf.Currency != 'NA').select(soDf["Currency"], soDf["CurrencyCode"]).show()

+--------------------+------------+
|            Currency|CurrencyCode|
+--------------------+------------+
|PKR\tPakistani rupee|         PKR|
|   EUR European Euro|         EUR|
|USD\tUnited State...|         USD|
|   EUR European Euro|         EUR|
| BRL\tBrazilian real|         BRL|
|   EUR European Euro|         EUR|
|   EUR European Euro|         EUR|
|   EUR European Euro|         EUR|
|USD\tUnited State...|         USD|
| GBP\tPound sterling|         GBP|
| GBP\tPound sterling|         GBP|
|USD\tUnited State...|         USD|
|   EUR European Euro|         EUR|
| GBP\tPound sterling|         GBP|
|   EUR European Euro|         EUR|
|USD\tUnited State...|         USD|
|   RON\tRomanian leu|         RON|
| GBP\tPound sterling|         GBP|
|   EUR European Euro|         EUR|
|USD\tUnited State...|         USD|
+--------------------+------------+
only showing top 20 rows



# Convert Compensation into GBP #

In [5]:
from pyspark.sql import functions as F
from pyspark.sql.types import DecimalType

# Join soDf to currencyConvDf where currencyConvDf.CurrencyCode = soDf.Currency Code
conversionDf = soDf.join(
    currencyConvDf,
    soDf["CurrencyCode"] == currencyConvDf["Currency Code"],
    "inner"
). \
    select(soDf["Country"], \
           soDf["CompTotal"], \
           soDf["LanguageHaveWorkedWith"], \
           currencyConvDf["Sterling value of Currency Unit £"])

# convert total compensation into GBP
conversionDf = conversionDf.withColumn("CompTotalInGBP", 
                    F.col("CompTotal") * F.col("Sterling value of Currency Unit £"))


# Split the LanguagesWorkedWith column on the ; character and explode it
explodedDf = conversionDf.withColumn(
    "Language",
    F.explode(F.split(F.col("LanguageHaveWorkedWith"), ";"))
)

explodedDf.show()


+--------------------+---------+----------------------+---------------------------------+--------------+--------------------+
|             Country|CompTotal|LanguageHaveWorkedWith|Sterling value of Currency Unit £|CompTotalInGBP|            Language|
+--------------------+---------+----------------------+---------------------------------+--------------+--------------------+
|            Pakistan|  2040000|  Assembly;Bash/She...|                           0.0028|        5712.0|            Assembly|
|            Pakistan|  2040000|  Assembly;Bash/She...|                           0.0028|        5712.0|Bash/Shell (all s...|
|            Pakistan|  2040000|  Assembly;Bash/She...|                           0.0028|        5712.0|                   C|
|            Pakistan|  2040000|  Assembly;Bash/She...|                           0.0028|        5712.0|                 C++|
|            Pakistan|  2040000|  Assembly;Bash/She...|                           0.0028|        5712.0|            HT

# And the winner is... #

In [7]:

# Filter out rows where CompTotalInGBP is 0 or null
filteredDf = explodedDf.filter((F.col("CompTotalInGBP") != 0) \
                               & (F.col("CompTotalInGBP").isNotNull()) 
                               )   

# Aggregate median CompTotalInGBP by Language and show the top 10
aggregatedDf = filteredDf.groupBy("Language").\
                agg(F.median("CompTotalInGBP").alias("Median Compensation In GBP")).\
                        orderBy(F.desc("Median Compensation In GBP")).\
                            limit(20)




In [8]:
aggregatedDf.show()

+--------------------+--------------------------+
|            Language|Median Compensation In GBP|
+--------------------+--------------------------+
|              Erlang|                   76627.2|
|             Clojure|                   75888.7|
|              Elixir|                   74000.0|
|                Ruby|                   73987.5|
|                Perl|         73434.40000000001|
|               Scala|                71476.6804|
|             Fortran|                   69304.0|
|                Lisp|                   67847.0|
|                  F#|                   64972.5|
|                  Go|                   63856.0|
|                Rust|         63562.72639999999|
|                Apex|                   63049.8|
|         Objective-C|                   62582.4|
|Bash/Shell (all s...|                   62373.6|
|              Groovy|                   62373.6|
|               Swift|        61507.299999999996|
|         MicroPython|                   60641.0|


In [9]:
# Filter out rows where CompTotalInGBP is 0 or null
filteredDf = explodedDf.filter((F.col("CompTotalInGBP") != 0) \
                               & (F.col("CompTotalInGBP").isNotNull()) \
                                   & (F.col("Country") == "United Kingdom of Great Britain and Northern Ireland"))   

# Aggregate median CompTotalInGBP by Language and show the top 10
aggregatedDf = filteredDf.groupBy("Language").\
                agg(F.median("CompTotalInGBP").alias("Median Compensation In GBP")).\
                        orderBy(F.desc("Median Compensation In GBP")).\
                            limit(20)

aggregatedDf.show()

+--------------------+--------------------------+
|            Language|Median Compensation In GBP|
+--------------------+--------------------------+
|               Swift|                  143676.0|
|                Java|                  119730.0|
|               Julia|                  119730.0|
|              Elixir|                  105362.4|
|         MicroPython|                  103956.0|
|              Kotlin|                  101504.0|
|                 C++|                  101504.0|
|            GDScript|                  101504.0|
|                  C#|                   86630.0|
|Bash/Shell (all s...|                   86630.0|
|                 PHP|                   83411.9|
|          JavaScript|                   83225.0|
|              Python|                   79820.0|
|                  Go|                   77142.0|
|          TypeScript|                   73434.4|
|            HTML/CSS|                   73434.4|
|                Dart|                   73434.4|
